In [4]:
pip install openai

In [5]:
pip install --upgrade typing-extensions

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install mysql-connector-python

In [8]:
from openai import OpenAI
import mysql.connector
import re

# MySQL 서버에 연결
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="0000",
    database="news"
)

# MySQL 커서 생성
cursor = conn.cursor()

# SQL 쿼리 실행
query = "SELECT article_id, content FROM article where article_id=42"
cursor.execute(query)

# 결과 가져오기
result = cursor.fetchall()

# GPT API 키 설정
client = OpenAI(api_key="OpenAi gpt-key",)

# 결과 출력
index = 0
while index < len(result):
    keyword_set = set()  # 중복을 확인하기 위한 set
    keyword_str = [None] * 10

    row = result[index]
    article_id, content = row
    print(f"ID: {article_id}")

    # 기사 내용과 질문을 합침
    content = content + "위의 기사에서 핵심 키워드 10개를 중복되지 않게 알려줘. 대답은 콤마로 구분해서 한줄로 해줘. 1. 키워드에 홑따옴표 제외하고 대답하기 2. 키워드에 가운뎃점 제외하고 대답하기 3. 키워드는 명사로만 대답하기"
   
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": content,
            }
        ],
        model="gpt-4",
    )
    message = chat_completion.choices[0].message  # 응답에서 메시지를 가져옵니다.
    print ("====>", message.content)  # GPT 응답 
    
    # 메시지에서 키워드 추출
    keywords = message.content
    # keywords = re.sub(r'\d+\.\s', '', keywords)  # 숫자와 점 다음의 공백을 제거

    # 줄 바꿈 문자를 기준으로 문자열 분할
    keywords = keywords.split(', ')

    for i in keywords[:10]:
        if i in keyword_set:  # 중복 확인
            print(f"Duplicate found: {i}. Restarting loop for current row.")
            index -= 1  # 현재 문 다시 시작
            break
        keyword_set.add(i)  # set에 추가
        # print(i)
    for i in keywords[:10]:
        # 여기에서 MySQL에 키워드를 추가
        # MySQL 쿼리를 사용하여 articlekeyword 테이블에 데이터를 추가
        insert_query = "INSERT INTO articlekeyword (keyword_id, article_id) VALUES (%s, %s)"
        cursor.execute(insert_query, (i, article_id))
        conn.commit()
        

    index += 1

# 연결 종료
cursor.close()
conn.close()

ID: 42
====> 날씨, 겨울, 기온, 한파, 지구온난화, 패딩, 단풍, 예보, 추위, 엘니뇨
